In [2]:
import pandas as pd

median = pd.read_pickle("model/model_median.pkl")

In [3]:
median

,SIT_UP,SIT_FLEX,LONG_RUN,RUN_10M,LONG_JUMP
F,22,33.5,18.8,12.535,160
M,42,45.0,11.7,10.680,212


In [1]:
median.iloc[0, 4]

NameError: name 'median' is not defined

In [24]:
from glob import glob
from os.path import isdir

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras import models
from keras import layers

# 1. 데이터 불러오기
# ====================================================

#데이터셋 폴더 내의 모든 csv 파일 경로를 file_list에 저장
file_list = []
for x in glob('./fithe/*.csv'):
    file_list.append(x)
file_list.sort()

# 메모리 용량 부족으로 일부 csv 파일만 선택할 때
file_list_small = file_list[-48:]

# 데이터셋 폴더 내의 모든 csv 파일을 읽어들여 하나로 합친다.
full_dataset = pd.DataFrame()

for file in tqdm(file_list):
     temp_dataset = pd.read_csv(file, sep=',')
     full_dataset = pd.concat([full_dataset, temp_dataset], axis=0)

full_dataset_r = full_dataset.reset_index(drop=True)
# print(full_dataset_r)
full_dataset_r.to_csv('./fithe/result.csv',encoding='utf-8')

dataset = pd.DataFrame()
dataset = pd.read_csv("fithe/result.csv", sep =',', encoding='utf-8')
print(dataset)
# #인덱스 배열 초기화
dataset_r = dataset.reset_index(drop = True, )

dataset_r.loc[(dataset_r.TEST_AGE >=10) & (dataset_r['TEST_AGE'] < 20),'TEST_AGE'] = 10
dataset_r.loc[(dataset_r.TEST_AGE >=20) & (dataset_r['TEST_AGE'] < 30),'TEST_AGE'] = 20
dataset_r.loc[(dataset_r.TEST_AGE >=30) & (dataset_r['TEST_AGE'] < 40),'TEST_AGE'] = 30
dataset_r.loc[(dataset_r.TEST_AGE >=40) & (dataset_r['TEST_AGE'] < 50),'TEST_AGE'] = 40
dataset_r.loc[(dataset_r.TEST_AGE >=50) & (dataset_r['TEST_AGE'] < 60),'TEST_AGE'] = 50
dataset_r.loc[(dataset_r.TEST_AGE >=60) & (dataset_r['TEST_AGE'] < 70),'TEST_AGE'] = 60
dataset_r.loc[(dataset_r.TEST_AGE >=70) & (dataset_r['TEST_AGE'] < 80),'TEST_AGE'] = 70
dataset_r.loc[(dataset_r.TEST_AGE >=80) & (dataset_r['TEST_AGE'] < 90),'TEST_AGE'] = 80
print(dataset_r['TEST_AGE'])


# # print(dataset_r)
# # 성인/청소년/노년층 데이터셋으로 분할

# qq = dataset_r.columns
# qq1 = dataset_r.info()
#
# q10m = dataset_r[(dataset_r['TEST_AGE'] >= 10) & (dataset_r['TEST_AGE'] < 20) & (dataset_r['TEST_SEX'] == 'M')]
# q10f = dataset_r[(dataset_r['TEST_AGE'] >= 10) & (dataset_r['TEST_AGE'] < 20) & (dataset_r['TEST_SEX'] == 'F')]
# q20m = dataset_r[(dataset_r['TEST_AGE'] >= 20) & (dataset_r['TEST_AGE'] < 30) & (dataset_r['TEST_SEX'] == 'M')]
# q20f = dataset_r[(dataset_r['TEST_AGE'] >= 20) & (dataset_r['TEST_AGE'] < 30) & (dataset_r['TEST_SEX'] == 'F')]
# q30m = dataset_r[(dataset_r['TEST_AGE'] >= 30) & (dataset_r['TEST_AGE'] < 40) & (dataset_r['TEST_SEX'] == 'M')]
# q30f = dataset_r[(dataset_r['TEST_AGE'] >= 30) & (dataset_r['TEST_AGE'] < 40) & (dataset_r['TEST_SEX'] == 'F')]
# q40m = dataset_r[(dataset_r['TEST_AGE'] >= 40) & (dataset_r['TEST_AGE'] < 50) & (dataset_r['TEST_SEX'] == 'M')]
# q40f = dataset_r[(dataset_r['TEST_AGE'] >= 40) & (dataset_r['TEST_AGE'] < 50) & (dataset_r['TEST_SEX'] == 'F')]
# q50m = dataset_r[(dataset_r['TEST_AGE'] >= 50) & (dataset_r['TEST_AGE'] < 60) & (dataset_r['TEST_SEX'] == 'M')]
# q60f = dataset_r[(dataset_r['TEST_AGE'] >= 60) & (dataset_r['TEST_AGE'] < 70) & (dataset_r['TEST_SEX'] == 'F')]
# q10m = dataset_r[(dataset_r['TEST_SEX'] == 'M')]
# q10f = dataset_r[(dataset_r['TEST_SEX'] == 'F')]
# adult_full = dataset_r[dataset_r['TEST_AGE']==20]
# adult_full = dataset_r[dataset_r['AGE_GBN']=='성인']
# youth_full = dataset_r[dataset_r['AGE_GBN']=='청소년']
# elder_full = dataset_r[dataset_r['AGE_GBN']=='노인']
# print(adult_full.head())

# print(q10m)
# print(q10f)
# print(q20)
# print(q30)
# print(q40)
# print(q50)
# print(q60)

column_list = [
    # 센터명, 측정 나이, 상장구분, 측정일, 성별
   'TEST_AGE','TEST_SEX']

m_col = [
    'ITEM_F001', #신장
    'ITEM_F002', #체중
    'ITEM_F003', #체지방율
    'ITEM_F004', #허리둘레
     # 건강체력항목
    'ITEM_F020', #20m왕복 오래달리기 (심폐지구력)
    'ITEM_F019', #교차윗몸일으키기 (근지구력)
    'ITEM_F012', #앉아윗몸앞으로굽히기 (유연성)
    'ITEM_F021', #10M 4회 왕복달리기(민첩성)
    'ITEM_F022', #제자리멀리뛰기(순발력)
            ]

g_col = [
    'ITEM_F001', #신장
    'ITEM_F002', #체중
    'ITEM_F003', #체지방율
    'ITEM_F004', #허리둘레
     # 건강체력항목
    'ITEM_F020', #20m왕복 오래달리기 (심폐지구력)
    'ITEM_F019', #교차윗몸일으키기 (근지구력)
    'ITEM_F012', #앉아윗몸앞으로굽히기 (유연성)
    'ITEM_F021', #10M 4회 왕복달리기(민첩성)
    'ITEM_F022', #제자리멀리뛰기(순발력)
            ]

m_col = column_list + m_col
g_col = column_list + g_col

m_dataset = dataset_r[dataset_r['TEST_SEX']=='M'][m_col].reset_index(drop = True)
g_dataset = dataset_r[dataset_r['TEST_SEX']=='F'][g_col].reset_index(drop = True)
print("m_dataset >>>>>>>>>>>>>>>>>>>",m_dataset)
# print("남자 : ",len(m_dataset))
# print("여자 : ",len(g_dataset))

# 결측치 제거 후 인덱스 리셋
m_dataset = m_dataset.dropna(axis=0).reset_index(drop = True)
g_dataset = g_dataset.dropna(axis=0).reset_index(drop = True)

print("남자 : ",len(m_dataset))
print("여자 : ",len(g_dataset))

# 컬럼명 재설정
m_dataset.columns = ['TEST_AGE', 'TEST_SEX',
                         'HEIGHT', 'WEIGHT', 'BODY_FAT', 'HURI_DUL', 'SIT_UP', 'SIT_FLEX', 'LONG_RUN','RUN_10M','LONG_JUMP']

g_dataset.columns = ['TEST_AGE', 'TEST_SEX',
                         'HEIGHT', 'WEIGHT', 'BODY_FAT', 'HURI_DUL', 'SIT_UP', 'SIT_FLEX', 'LONG_RUN','RUN_10M','LONG_JUMP']


mode = 'M' # M OR F

# 모델 학습 모드에 맞게 데이터셋 선택
if mode == 'M' : dataset = m_dataset
else : dataset = g_dataset

# TRAIN TEST SPLIT
# train : 80% / test : 20%
test_idx = len(dataset)//10
train_dataset = dataset[:-(test_idx*2)].reset_index(drop=True)
test_dataset = dataset[(test_idx*2):].reset_index(drop=True)
len(dataset)//10

print(train_dataset.head())

# 이상값 제거 (+-1.5 IQR 범위 )
first_q = train_dataset.iloc[:,9:].quantile(q = 0.25)
third_q = train_dataset.iloc[:,9:].quantile(q = 0.75)
IQR = third_q - first_q
bottom = first_q - 1.5 *IQR
top = third_q + 1.5*IQR
print("top :>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> : ",top) # 이상값 기준

# top 이상일 때 top으로 조정
for column in top.index :
    train_dataset[column] = train_dataset[column].apply(lambda x: top[column] if x > top[column] else x)


train_dataset = train_dataset.dropna()
# 범주형 변수 더미화
train_dataset['TEST_SEX'] = train_dataset['TEST_SEX'].apply(lambda x: 1 if x == 'M' else 0)

# 체지방률 예측을 위해 y를 체지방률로 둔다.
train_dataset_X = train_dataset.drop(columns=['TEST_AGE','BODY_FAT'],axis = 1)
train_dataset_y = train_dataset['TEST_AGE']

from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
# y = encoder.fit_transform(train_dataset_y)

# 연속형 변수 스케일링(MinMaxScaler)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_dataset_X)
X_train_scaled = scaler.transform(train_dataset_X)

# 학습 / 검증 데이터셋 분리
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, train_dataset_y,
                                                    test_size=0.3,
                                                    random_state=1,
                                                    shuffle = True,
                                                    stratify = train_dataset_X['TEST_SEX'])


print('X_train shape:', X_train.shape)

print('X_val shape:', X_val.shape)

print('y_train shape:', y_train.shape)

print('y_val shape:', y_val.shape)


# ====================================================
# 3. 모델 학습
# ====================================================
from keras.optimizers import Adam
Adams = Adam(lr=0.005)

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(40, activation='relu'))
    model.add(layers.Dense(14, activation='relu'))#32
    model.add(layers.Dense(1))

    model.compile(optimizer=Adams, loss='mse', metrics=['mae'])

    return model

from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience = 30)
print(X_train_scaled)

model = build_model()
model.fit(X_train, y_train,
          validation_data=(X_val, y_val),
          epochs=50, batch_size=24,verbose=1,callbacks=[early_stopping])

# ====================================================
# 4. 모델 평가
# ====================================================
test_dataset_ = test_dataset

# 범주형 더미화
test_dataset_['TEST_SEX'] = test_dataset_['TEST_SEX'].apply(lambda x: 1 if x == 'M' else 0)

# m_dataset.columns = ['TEST_AGE', 'TEST_SEX',
#                          'HEIGHT', 'WEIGHT', 'BODY_FAT', 'HURI_DUL', 'SIT_UP', 'SIT_FLEX', 'LONG_RUN','RUN_10M','LONG_JUMP']
# 'ITEM_F020',  # 20m왕복 오래달리기 (심폐지구력)
# 'ITEM_F019',  # 교차윗몸일으키기 (근지구력)
# 'ITEM_F012',  # 앉아윗몸앞으로굽히기 (유연성)
# 'ITEM_F021',  # 10M 4회 왕복달리기(민첩성)
# 'ITEM_F022',  # 제자리멀리뛰기(순발력)
# 체지방률 예측을 위해 y를 체지방률로 둔다.
test_dataset_X = test_dataset_.drop(columns=['TEST_AGE','BODY_FAT'],axis = 1)
test_dataset_y = test_dataset_['TEST_AGE']
sex = test_dataset_['TEST_SEX'] = 0
hei = test_dataset_['HEIGHT'] = 167.5
wei = test_dataset_['WEIGHT'] = 68.5
hur = test_dataset_['HURI_DUL'] = 90.8
up = test_dataset_['SIT_UP'] = 4
flex = test_dataset_['SIT_FLEX'] = 5
long = test_dataset_['LONG_RUN'] = 19
run = test_dataset_['RUN_10M'] = 12.66
jump = test_dataset_['LONG_JUMP'] = 180
test_data = [[sex, hei, wei, hur, up, flex, long, run, jump]]

# print(test_dataset_X)
X_test_scaled = scaler.transform(test_dataset_X)
print("X_test_scaled",X_test_scaled)
predict = model.predict(X_test_scaled)

print(model.evaluate(X_test_scaled,test_dataset_y))

# # 모델 저장
# model_num = '1208'
# model.save(mode+'_model_'+model_num+'.h5')
#
# import joblib
# joblib.dump(scaler,mode+'_scaler_'+model_num+'.pkl')

ModuleNotFoundError: No module named 'sklearn'

In [2]:
import pandas as pd

median = pd.read_pickle("model/model_median.pkl")

In [3]:
median

,SIT_UP,SIT_FLEX,LONG_RUN,RUN_10M,LONG_JUMP
F,22,33.5,18.8,12.535,160
M,42,45.0,11.7,10.680,212


In [14]:
median

16.0

In [13]:
median.iloc[1, 4]

AttributeError: 'numpy.float64' object has no attribute 'iloc'